In [1]:
# !pip install git+https://github.com/kubeflow/sdk.git@main

In [2]:
from kubeflow.trainer import CustomTrainer, TrainerClient

client = TrainerClient()

In [3]:
from kubeflow.trainer import Runtime, RuntimeTrainer, TrainerType

# Create custom Runtime
custom_runtime = Runtime(
    name="custom-pytorch-runtime",
    trainer=RuntimeTrainer(
        trainer_type=TrainerType.CUSTOM_TRAINER,
        framework="torch",
        num_nodes=2,
    ),
    pretrained_model=None
)



In [4]:
for runtime in client.list_runtimes():
    print(runtime)
    if runtime.name == "torch-distributed-speech-recognition":
        torch_runtime = runtime


Runtime(name='torch-distributed-speech-recognition', trainer=RuntimeTrainer(trainer_type=<TrainerType.CUSTOM_TRAINER: 'CustomTrainer'>, framework='torch', num_nodes=1, device='Unknown', device_count='Unknown'), pretrained_model=None)
Runtime(name='torch-distributed-voice-recognition', trainer=RuntimeTrainer(trainer_type=<TrainerType.CUSTOM_TRAINER: 'CustomTrainer'>, framework='torch', num_nodes=1, device='Unknown', device_count='Unknown'), pretrained_model=None)


In [6]:
import train_with_kubeflow_trainer

job_name = client.train(
    trainer=CustomTrainer(
        func=train_with_kubeflow_trainer.run,
        # Set how many PyTorch nodes you want to use for distributed training.
        num_nodes=2,
        # Set the resources for each PyTorch node.
        resources_per_node={
            "cpu": 2,
            "memory": "8Gi",
            # Uncomment this to distribute the TrainJob using GPU nodes.
            # "nvidia.com/gpu": 1,
        },
    ),
    runtime=torch_runtime,
)

In [6]:
client.wait_for_job_status(name=job_name, status={"Running"})




TrainJob(name='m1b42f2aab8e', creation_timestamp=datetime.datetime(2025, 9, 9, 6, 7, 17, tzinfo=TzInfo(UTC)), runtime=Runtime(name='torch-distributed-voice-recognition', trainer=RuntimeTrainer(trainer_type=<TrainerType.CUSTOM_TRAINER: 'CustomTrainer'>, framework='torch', num_nodes=1, device='Unknown', device_count='Unknown'), pretrained_model=None), steps=[Step(name='node-0', status='Running', pod_name='m1b42f2aab8e-node-0-0-hxsmv', device='cpu', device_count='4'), Step(name='node-1', status='Running', pod_name='m1b42f2aab8e-node-0-1-k869r', device='cpu', device_count='4'), Step(name='node-2', status='Running', pod_name='m1b42f2aab8e-node-0-2-xtz4g', device='cpu', device_count='4')], num_nodes=3, status='Running')